---
title: EDA - Wavelength Distribution
project: "dataset_EDA"
cdt: 2024-09-06T13:16:52
description: "Observation of wavelength distribution and determination of common cutoff"
---


In [ ]:
import duckdb as db
import polars as pl

db_path = "/Users/jonathan/mres_thesis/wine_analysis_hplc_uv/wines.db"

con = db.connect(db_path)


What are the unique counts of sample per wavelength?

In [ ]:
# number of unique counts

samples_per_wavelength = con.sql(
    """
    SELECT
        wavelength,
        COUNT(DISTINCT ID) as num_samples
    FROM
        pbl.chromatogram_spectra_long
    GROUP BY
        wavelength
    ORDER BY
        num_samples DESC
    """
)

unique_sample_counts = con.sql(
    """
    SELECT
        DISTINCT num_samples as unique_sample_counts
    FROM
        samples_per_wavelength
    """
)

unique_sample_counts.pl()


Not sure how to interpret that one, 71 samples have wavelengths on top of the that the other 104 dont have?

Yes. Every samples wavelength starts at 190nm.

In [ ]:
wavelength_min_max_per_id = con.sql(
    """
    SELECT
        id,
        min(wavelength) as nm_min,
        max(wavelength) as nm_max,
    FROM
        pbl.chromatogram_spectra_long
    GROUP BY
        id
    """
)
wavelength_min_max_per_id


does every sample start at 190?

In [ ]:
unique_min_wavelengths = con.sql(
    """
    SELECT
        DISTINCT nm_min
    FROM
        wavelength_min_max_per_id
    """
)
unique_min_wavelengths


how many end wavelengths?

In [ ]:
unique_max_wavelengths = con.sql(
    """
    SELECT
        DISTINCT nm_max
    FROM
        wavelength_min_max_per_id
    """
)
unique_max_wavelengths


there are two wavelengths, 600 and 400.

In [ ]:
# how many per wavelength range?

samples_per_wavelength_max = con.sql(
    """
    SELECT
        nm_max,
        COUNT (id)
    FROM
        wavelength_min_max_per_id
    GROUP BY
        nm_max
    ORDER BY
        nm_max
    """
)

samples_per_wavelength_max


104 


In [ ]:
def describe_wavelengths(con):
    query = \
    """
    SELECT
        id,
        min(wavelength),
        max(wavelength),
        count(wavelength)
    FROM
        pbl.chromatogram_spectra_long
    GROUP BY
        id
    """
    
    con.sql(query).pl()

describe_wavelengths(con)
    